# fx trading strategies in crypto space

- [introduction](#introduction)
- [assumptions](#assumptions)
- [carry?](#carry?)
- [how/what to trade](#how/what-to-trade)
- [return calculation](#return-calculation)
    - [long](#long)
    - [short](#short)
    - [long-short](#long-short)
- [results](#results)
    - [imports and parameters](#imports-and-parameters)
    - [data](#data)
        - [spot price](#spot-price)
        - [perpetual futures price](#perpetual-futures-price)
        - [funding rates](#funding-rates)
    - [carry](#carry)
    - [portfolio construction](#portfolio-construction)
    - [excess returns](#excess-returns)
    - [carry trade performance](#carry-trade-performance)

## introduction
Cryptocurrencies are quite rapidly becoming a fully-fledged investable asset. One can debate about the lack of fundamental value, questionable incentives of issuers and end users, uncertain regulatory future etc. &mdash; but as long as there is supply and demand, an open market, derivative products and sufficient liquidity, Bitcoin and its peers are just as assety as stocks, bonds and commodity products.

Existence of a derivatives market, specifically futures, calls forth the concept of carry and provides a way to quantify it. In this short piece, I am using perpetual swaps, funding rates and spot prices from the cryptocurrency exchange Kraken to construct a simple long-short carry trade strategy on the set of five most liquid cryprocurrencies and investigate its properties. 

## literature
As of the latest commit, the only relevant work on the same topic is ["Crypto Carry"](https://dx.doi.org/10.2139/ssrn.3774118), which documents return predictability of the funding rate in the cross-section of 51 coins listed on the Huobi Exchange and a Sharpe ratio of the corresponding long-short strategy of 1.21. However, the authors are silent about the methodology of return computations. Our contribution &mdash; without pretending to be original &mdash; is to provide a fully transparent and replicable open source research on the existence of carry trade on the cryptocurrency markets.

## assumptions
1. USD is the currency of consumption;
1. trading takes place on Kraken;
1. investors can trade at mid prices;
1. investors do not move the market;
1. costs of setting up a crypto wallet and depositing/withdrawing liquidity are zero;
1. spot selling is at zero additional cost in terms of money and time.

Assumptions 1-3 are boring; assumption 4 is violated the more, the **larger** the trading volume [*TODO: references about trade sizes?*], and assumption 5 is violated the more, the **smaller** the trading volume; assumption 6 is violated regardless.

With these assumptions in place, I can answer the question whether the carry trade is there but not whether it is certainly economically profitable to execute. Still fun.

## carry?
"Carry trade" is a strategy for an investor who hopes that the spot price does not move over the investment horizon. It is constructed by taking long forward positions in assets with high, and short &mdash; in those with low "carry". The "carry" is the difference between the currently observed spot and futures prices (expressed as a percentage of the former), and is related to the opportunity costs and the costs of holding ("carrying") the asset. In the case of the cryptocurrency perpetual contracts, the funding rate is a related concept, and we will be using it instead of the very noisy price difference for the signal construction and subsequent sorting purposes. More about the funding rates follows.

## how/what to trade
The most liquid contracts to implement strategies with are the so-called [perpetual futures](https://support.kraken.com/hc/en-us/articles/360022835911-Inverse-crypto-collateral-perpetual-contract-specifications): these can be thought of as 4-hour swaps that are automatically rolled over earning what is called the [funding rate](https://blog.kraken.com/post/3075/quick-primer-on-funding-rates/https://blog.kraken.com/post/3075/quick-primer-on-funding-rates/). The latter is not dissimilar to the interest earned by the holder of a long ForEx position: longs pay shorts a certain amount which is determined based on the difference between the recent futures and spot prices (remember the carry concept?). If the futures has been traded too high relative to the spot, the funding rate becomes high also, incentivising market participants to take short positions (to be able to earn this high rate) and thus driving the futures price back down. 

Probably because financial intermediaries shun away from clearing  transactions involving fiat and cryptocurrencies, there are no familiar analogues to USDRUB, AUDUSD etc; it is not common to put USD as collateral to trade futures and realize PnL in dollars. Two alternatives exist: linear futures with stablecoins as the counter currency (such as Binance futures on ETHUSDT) and inverse futures margined in cryptocurrency (such as Kraken futures on ETHUSD). 

The former are the more familiar contracts as far as the specifications and PnL calculation are concerned, but require to purchase the countercurrency coins first, such as the Tether (USDT) or USD coin (USDC) -- two of the so-called stablecoins. A standard trade would go as follows:
1. buy USDT for USD on the spot market;
1. use USDT in the margin account to collateralize a position in ETHUSDT, BTCUSDT etc.;
1. realize PnL in USDT;
1. sell USDT for USD to consume.

An obvious disadvantage is the additional USD-USDT risk.

Inverse crypto-collateral futures are essentialy linear futures on the inverse of the pair. They seem to be more cost-efficient and do not involved a third currency, but contain implicit leverage for an investor with consumption denominated in the fiat currency (details later). A standard trade would go as follows:
1. buy the contract margin cryptocurrency for USD on the spot market;
1. use it to collateralize a position in the futures on the same cryptocurrency;
1. realize PnL in the cryptocurrency;
1. sell the cryptocurrency for USD to consume.

In what follows, I will use the inverse futures.

## return calculation
I use the following conventions:

* $S$ : spot exchange rate, denominated in USD;
* $F$ : futures rate, denominated in USD;
* $q$ : absolute funding rate, in coins per contract;
* $N_i$ : number of contracts of side $i, i \in (l, s)$ for (long, short);
* $r$ : net return, in fractions of 1, such as 0.1 for 10%

At time $t$ we start with 1 USD. Since the contracts are coin-margined, we need to convert this 1 USD into coins at rate $S_t$ and use it as collateral for all positions we will open. On most exchanges one contract is worth 1 USD and hence represents $\frac{1}{F_t}$ coins at inception.

**Note that in the empyrical part of this piece we do not reinvest the proceeds, starting each period with 1 dollar to collateralize the positions.**

### long

Being long $N_l$ inverse futures at rate $F_t$ is essentially being short USD at rate $\frac{1}{F_t}$, hence the payoff at time $t+1$ is $\left( \frac{1}{F_t} - \frac{1}{F_{t+1}} \right)$ coins. So, at $t+1$ we have a total of $N_l \left( \frac{1}{F_t} - \frac{1}{F_{t+1}}\right)$ coins in trading profit plus $\frac{1}{S_t}$ coins in the collateral account, all of which can be converted back at the prevailing rate for the total USD profit of:
\begin{equation*}
    N_l \left( \frac{1}{F_t} - \frac{1}{F_{t+1}}\right)S_{t+1} + \frac{S_{t+1}}{S_t}.
\end{equation*}
Now, assuming that $S \approx F$ and using $r = \frac{S_{t+1}}{S_t} - 1$, the above amount is equal to:
\begin{equation}
    1 + r(N_l + 1)
    \label{gross_ret_simplified}\tag{1}
\end{equation}
Since we started with 1 USD, this is also the gross return on our investment. Behold the implicit leverage: if we are long (the obvious minimum of) 1 contract, our return is twice the spot return! 

The final thing to take account of is the funding rate: being long, we pay the funding rate of $q_t$ coins per contract (note that this is the "absolute" funding rate, not the relative counterpart displayed at the front end of the exchanges). The exact gross return formula for a long position becomes:
\begin{equation}
    N_l \left( \frac{1}{F_t} - \frac{1}{F_{t+1}} - q_t \right) S_{t+1} + 
    \frac{S_{t+1}}{S_t}.
\end{equation}


### short

Being short $N_s$ inverse futures at rate $F_t$ is essentially being long USD at rate $\frac{1}{F_t}$. Similar considerations lead to the following equivalent of eq. [1](#mjx-eqn-gross_ret_simplified):
\begin{equation}
    1 + r(1 - N_s).
\end{equation}
Curiously, a zero-leverage short position of 1 contract *cannot grow or drop* in value in USD terms! If the coin price falls, we have more coins, which are worth less, and also the collateral account loses USD -- these forces cancel each other out. To be able to lose or earn USD, leverage is needed.

Before we proceed, here is the exact gross return formula for a short position:
\begin{equation}
    N_s \left( \frac{1}{F_{t+1}} - \frac{1}{F_t} + q_t \right) S_{t+1} + 
    \frac{S_{t+1}}{S_t}.
\end{equation}

### long-short


In a long-short portfolio, to make the long leg have the same leverage as the short, we can set $N_l=1$ and $N_s=3$, resulting in a multiplier of 2 on each. Since one contract is worth 1 USD, and we start with 1 USD in the collateral account, we have to open a total of 4 USD worth of contracts having 1 USD to collateralize them. Most exchanges would allow for much higher ratios.

As an example, imagine that the price of the coins in the long and short leg increases by 15\% and 5\% respectively: the portfolio return in this case (disregarding the tiny differences between $F$ and $S$ as well the funding rates) is $2(0.15 - 0.05)=20\%$.

## results

### imports and parameters

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt''),
import seaborn as sns
from matplotlib.dates import YearLocator, MonthLocator, DateFormatter

from foolbox.econometrics.misc import descriptives
from foolbox.backtesting.sorting import rank_sort

from src.setup import *

from src.datafeed_.kraken.downstream import (get_funding_rates, 
                                             get_spot, 
                                             get_perpetual)

In [ ]:
# pandas display settings
pd.set_option("display.precision", 2)

# parameters
t_hold = 1  # holding period, in multiples of a 4-hour period
ann = 365 * 6  # there are 6 4-hour periods per day
legsize = 2  # number of assets in the long and short leg of an HML portfolio
signal_lookback = 6 * 7  # 7 days lookback to compute carry signal

def format_dataframe(df_: pd.DataFrame, tail=True):
    if tail:
        return format_dataframe(df_.tail(), tail=False)
    if df_.columns.nlevels > 1:
        col_key = ("timestamp", "")
    else:
        col_key = "timestamp"
    res_ = df_.reset_index()\
        .style.format(formatter={col_key: lambda x: x.strftime("%Y-%m-%d %H:%M")})\
        .hide_index()
    return res_

### data
All data is from Kraken, snapped at 4 hour intervals starting at 00:00.

#### spot price
Let's take a look at the spot prices going through the roof.

In [ ]:
# spot price
data_s = get_spot()\
    .resample("4H", closed="right", label="right").last()\
    .loc["2018-08":]
format_dataframe(data_s)

In [ ]:
# plot spot price index
spot_idx = data_s.div(data_s.iloc[0], axis=1)
ax = sns.lineplot(data=spot_idx, dashes=False)

# formatting
ax.set_xticks(
    pd.period_range("2018-09", "2021-03", freq="6M").to_timestamp()
)
ax.xaxis.set_major_locator(YearLocator(1))
ax.xaxis.set_minor_locator(MonthLocator([4, 7, 10]))
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
ax.xaxis.set_minor_formatter(DateFormatter("%b"))

ax.grid(which="both")

ax.set_title("cryptocurrency price dynamics")

ax.set_ylabel("index (08/2018 = 1)")
ax.set_xlabel("", visible=False)

plt.show()

#### perpetual futures price

In [ ]:
# load perpetual futures prices, reindex as the spot
data_p = get_perpetual(mid=True) \
    .resample("4H", closed="right", label="right").last()\
    .reindex_like(data_s)

format_dataframe(data_p)

Let's take a look at how close the perpetual is to the spot for the case of, say, XBT, zooming in on just one day.

In [ ]:
# take these for the example
c_ = "xbt"
d_ = "2021-08-15"

# plot two prices
to_plot = pd.concat((data_s[c_], data_p[c_]), axis=1, 
                    keys=["spot", "perpetual"])\
    .loc[d_]

ax = sns.lineplot(data=to_plot, dashes=False)

ax.xaxis.set_major_formatter(DateFormatter("%H"))

ax.grid(which="both")

ax.set_title(f"{c_} spot vs perpetual price")

ax.set_ylabel("price, in USD")
ax.set_xlabel(f"hour of {d_}", visible=True)

plt.show()

#### funding rates

In [ ]:
# abs and relative funding rate
data_f = get_funding_rates()
format_dataframe(data_f)

### carry

As mentioned before, the carry of a futures contract is the difference between the spot and futures prices, expressed as the percentage of the former. However, this measure is quite noisy at the 4-hour frequency, so for the signal construction purposes here we rely on a similar concept of the funding rate, which is nothing but a trimmed time-series average of the actual carry.

In [ ]:
# carry is the negative of the relative funding rate, ~(spot / fwd - 1)
carry = data_f \
    .query("which == 'relative'").drop("which", axis=1) \
    .pivot(index="timestamp", columns="asset", values="rate")\
    .mul(-1)

Here is the spot/futures ratio less 1, also known as the forward discount, compared to the relative funding rate.

In [ ]:
fwd_d = (data_s / data_p - 1)

c_ = "xbt"
m_ = "2021-03"

to_plot = pd.concat((fwd_d.loc[m_, c_], carry.loc[m_, c_]), 
                    axis=1, keys=["fwd-disc", "rel-fund"]) * 10000

# plot on two axes, style
ax = sns.lineplot(data=to_plot["fwd-disc"], color=sns.color_palette("colorblind")[0])
ax2 = ax.twinx()
sns.lineplot(data=to_plot["rel-fund"], color=sns.color_palette("colorblind")[1], ax=ax2)
ax2.set_ylim(np.array(ax.get_ylim()) / 10)
ax2.set_yticks(ax.get_yticks() / 10)
ax2.set_ylim(np.array(ax.get_ylim()) / 10)

ax.xaxis.set_major_formatter(DateFormatter("%d"))

ax.grid(axis="both")

ax.set_title(f"measures of carry of {c_}")

ax.set_ylabel("forward discount, in pips")
ax2.set_ylabel("rel fund rate, in pips")
ax.set_xlabel(f"day of {m_}", visible=True)

plt.show()

Below is the average carry of cryptocurrency contracts and its descriptive statistics.

In [ ]:
# plot
ax = sns.barplot(data=carry * ann * 100)
ax.set_title("carry of cryptocurrency contracts")
ax.set_ylabel("percent p.a.")

plt.show()

In [ ]:
# print descriptives (some are meaningless)
carry_d = descriptives(carry, ann=ann, scl=100, cov_lags=30)

# print("carry of cryptocurrency contracts: descriptives\n")
# print(carry_d.iloc[:-2])
# print("-" * 41 + "\n")
# print("values are in percent p.a. where relevant.\n")
format_dataframe(carry_d, tail=False)

From this perspective, cryptocurrencies resemble fiat funding currencies such as JPY and EUR which also feature negative carry.

### portfolio construction

In [ ]:
# the signal is the rolling median of the carry (proxy)
carry_sig = carry\
    .rolling(signal_lookback, min_periods=signal_lookback // 2)\
    .median()\
    .shift(1)

# below, 1 means that the asset is in portfolio at time t;
# portfolios are in the top level of the columns, while assets
# are in the bottom level
sorts = rank_sort(carry_sig, legsize=legsize)

format_dataframe(sorts[["p_low", "p_high"]])

In [ ]:
# plot positions
ax = sorts["p_high"].add(sorts["p_low"].mul(-1)).plot(subplots=True)

plt.show()

It can be seen that cryptocurrencies move between portfolios a lot &mdash; in stark contrast to fiat currencies in a typical fiat FX carry strategy, where the high and low portfolios are relatively stable.

### excess returns
Here, we implement the formulas in section ["return calculation"](#return-calculation). To keep the same leverage of 2 in the long and short leg, we use 1 long contract and 3 short ones.

In [ ]:
# forward-looking returns and funding (levered x2)
# need 1 long and 3 short contracts to achieve the same level of leverage
n_long = 1
n_short = 3

# absolute funding rate
f_rate = data_f \
    .query("which == 'absolute'").drop("which", axis=1) \
    .pivot(index="timestamp", columns="asset", values="rate")

# long
r_long = n_long * data_s.shift(-t_hold) * \
     (1 / data_p - 1 / data_p.shift(-t_hold) -
      f_rate.shift(1).mul(4).rolling(t_hold).sum().shift(-t_hold + 1)) + \
     data_s.shift(-t_hold) / data_s - 1

# short
r_short = n_short * data_s.shift(-t_hold) * \
      (1 / data_p.shift(-t_hold) - 1 / data_p +
       f_rate.shift(1).mul(4).rolling(t_hold).sum().shift(-t_hold + 1)) + \
      data_s.shift(-t_hold) / data_s - 1

# both
rx = pd.concat([r_long, r_short], axis=1, keys=["p_high", "p_low"],
               names=["portfolio", "asset"])

print("excess returns, in percent")
format_dataframe(rx.dropna(how="all") * 100)

In [ ]:
# take a look at returns
print("p_high (x2 leverage):")
format_dataframe(descriptives(rx.xs("p_high", axis=1), ann=ann/t_hold, scl=100), tail=False)

In [ ]:
print("\np_low (x2 leverage):")
format_dataframe(descriptives(rx.xs("p_low", axis=1), ann=ann/t_hold, scl=100), tail=False)

### carry trade performance

All currencies receive the same weight of 1/legsize if they are in the long, and -1/legsize or short portfolio. The excess return on a +1 long or -3 short position is calculated by dividing `sorts` which keeps 1.0s nad 0.0s through `legsize` and multiplying with the returns from the previous cell. The high-minus-low portfolio is the sum of the two (not the difference by the way the returns on short positions are calculated).

In [ ]:
# sort returns into portfolios according to the signal
res_hl = rx.mul(sorts / legsize) \
    .groupby(axis=1, level="portfolio").sum(min_count=1) \
    .loc[:, ["p_high", "p_low"]].dropna()

# high minus low
res_hl.insert(0, "p_hml", res_hl.eval("p_high + p_low"))

format_dataframe(res_hl)

Let's calculate the descriptives and plot the cumulative returns.

In [ ]:
# means, st err etc.
rx_d = descriptives(res_hl, ann=(ann / t_hold), scl=100)

format_dataframe(rx_d, tail=False)

In [ ]:
# plot
to_plot = res_hl.cumsum()\
    .rename(columns=lambda x: x[2:])\
    .reset_index().melt(id_vars=["timestamp"], value_name="payoff")
ax = sns.lineplot(x="timestamp", y="payoff", data=to_plot, hue="portfolio")

ax.xaxis.set_major_locator(YearLocator(1))
ax.xaxis.set_minor_locator(MonthLocator([1, 4, 7, 10]))
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
ax.xaxis.set_minor_formatter(DateFormatter("%b"))
ax.grid(True, which="both")

ax.set_xlabel("", visible=False)
ax.set_ylabel("payoff w/o reinvestment, in USD")

plt.show()

The long portfolio over- and the short one underperforms. The high-low strategy has earned about 700% return (w/o reinvestment) since end of september 2019, with a Sharpe ratio of 1.84.

</walkthrough>